In [1]:
import os
import argparse
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
from torchvision.utils import save_image
import torch.nn.functional as F
import matplotlib.pyplot as plt
from PIL import Image
import inspect

from custom_dataset import CustomImageDataset

In [2]:
parser = argparse.ArgumentParser()
parser.add_argument("--n_epochs", type=int, default=2000, help="Number of epochs of training")
parser.add_argument("--batch_size", type=int, default=64, help="Size of the batches")
parser.add_argument("--lr", type=float, default=0.0002, help="Learning rate for the optimizer")
parser.add_argument("--b1", type=float, default=0.5, help="Beta1 parameter for the optimizer")
parser.add_argument("--b2", type=float, default=0.999, help="Beta2 parameter for the optimizer")
parser.add_argument("--n_cpu", type=int, default=8, help="Number of CPU threads for data loading")
parser.add_argument("--latent_dim", type=int, default=100, help="Dimensionality of the latent space")
parser.add_argument("--img_size", type=int, default=64, help="Size of each image dimension")
parser.add_argument("--channels", type=int, default=3, help="Number of image channels")
parser.add_argument("--sample_interval", type=int, default=10000, help="Interval between image samples")
parser.add_argument("--csv_file", type=str, default="female_mac.csv", help="CSV file containing image paths and labels")
opt = parser.parse_args(["--n_epochs", "10", "--batch_size", "32", "--lr", "0.0002", "--b1", "0.5",
                     "--b2", "0.999", "--n_cpu", "4", "--latent_dim", "100", "--img_size", "28",
                     "--channels" ,"1", "--sample_interval", "10000", "--csv_file", "female_mac.csv"])

In [3]:
opt.latent_dim

100

In [4]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()

        self.model = nn.Sequential(
            nn.ConvTranspose2d(opt.latent_dim, 512, kernel_size=4, stride=1, padding=0, bias=False),
            nn.BatchNorm2d(512),
            nn.ReLU(True),
            nn.ConvTranspose2d(512, 256, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(True),
            nn.ConvTranspose2d(256, 128, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(128),
            nn.ReLU(True),
            nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(True),
            nn.ConvTranspose2d(64, opt.channels, kernel_size=4, stride=2, padding=1, bias=False),
            nn.Tanh()
        )

    def forward(self, z):
        z = z.view(z.size(0), z.size(1), 1, 1)
        img = self.model(z)
        return img


class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        self.model = nn.Sequential(
            nn.Conv2d(opt.channels, 64, kernel_size=3, stride=1, padding=1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(256, 512, kernel_size=4, stride=2, padding=0, bias=False),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(512, 1, kernel_size=4, stride=1, padding=0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, img):
        validity = self.model(img)
        return validity


def adversarial_loss(output, target):
    target = target.expand_as(output)
    loss = F.binary_cross_entropy(output, target)
    return loss


def save_model(opt, generator, discriminator, optimizer_G, optimizer_D, current_epoch):
    out = os.path.join(opt.model_path, "checkpoinnt_{}.tar".format(current_epoch))
    state = {
        'generator': generator.state_dict(),
        'discriminator': discriminator.state_dict(),
        'optimizer_G': optimizer_G.state_dict(),
        'optimizeer_D': optimizer_D.state_dict(),
        'epoch': current_epoch
    }
    torch.save(state,out)

In [5]:
def train(opt):
    os.makedirs("images", exist_ok=True)

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    cid = CustomImageDataset(opt.csv_file, opt.img_size)

    dataloader = DataLoader(
        cid,
        batch_size=opt.batch_size,
        shuffle=True,
        drop_last=True,
        num_workers=opt.n_cpu,
    )

    generator = Generator().to(device)
    discriminator = Discriminator().to(device)

    cuda = torch.cuda.is_available()
    if cuda:
        generator.cuda()
        discriminator.cuda()

    optimizer_G = optim.Adam(generator.parameters(), lr=opt.lr, betas=(opt.b1, opt.b2))
    optimizer_D = optim.Adam(discriminator.parameters(), lr=opt.lr, betas=(opt.b1, opt.b2))

    Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor

    generator_losses = []
    discriminator_losses = []
    
#     print(inspect.getmembers(dataloader),'\n\n*********************************************************\n\n')

    print(hasattr(dataloader, 'num_workers'))
    print(dataloader.num_workers)
#     dataloader.num_workers = 4
    print(dataloader.num_workers)
#     print(type(dataloader))
#     print(dataloader.__main__.CustomImageDataset)

    for epoch in range(opt.n_epochs):
        for i, (imgs,_) in enumerate(dataloader):
            valid = Tensor(imgs.size(0), 1, 1, 1).fill_(1.0)
            fake = Tensor(imgs.size(0), 1, 1, 1).fill_(0.0)

            real_imgs = imgs.type(Tensor)

            optimizer_G.zero_grad()

            z = Tensor(np.random.normal(0, 1, (imgs.shape[0], opt.latent_dim)))

            gen_imgs = generator(z)

            g_loss = adversarial_loss(discriminator(gen_imgs), valid)

            g_loss.backward()
            optimizer_G.step()

            optimizer_D.zero_grad()

            real_loss = adversarial_loss(discriminator(real_imgs), valid)
            fake_loss = adversarial_loss(discriminator(gen_imgs.detach()), fake)
            # target = torch.ones_like(real_loss).to(device)

            d_loss = (real_loss + fake_loss) / 2

            d_loss.backward()
            optimizer_D.step()

            print("[Epoch %d/%d] [Batch %d/%d] [D loss: %.4f] [G loss: %.4f]"
                % (epoch, opt.n_epochs, i, len(dataloader), d_loss.item(), g_loss.item()))

            batches_done = epoch * len(dataloader) + i
            if batches_done % opt.sample_interval == 0:
                save_image(gen_imgs.data[:25], "images/%d.png" % batches_done, nrow=5, normalize=True)

        generator_losses.append(g_loss.item())
        discriminator_losses.append(d_loss.item())


    plt.plot(generator_losses, label='Generator Loss')
    plt.plot(discriminator_losses, label='Discriminator Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()
    plt.savefig('plot.png')

In [6]:
train(opt)

True
4
4


AttributeError: Caught AttributeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/Users/cristianespinosa/opt/anaconda3/envs/tensor/lib/python3.9/site-packages/PIL/Image.py", line 3096, in open
    fp.seek(0)
AttributeError: 'numpy.int64' object has no attribute 'seek'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/cristianespinosa/opt/anaconda3/envs/tensor/lib/python3.9/site-packages/torch/utils/data/_utils/worker.py", line 302, in _worker_loop
    data = fetcher.fetch(index)
  File "/Users/cristianespinosa/opt/anaconda3/envs/tensor/lib/python3.9/site-packages/torch/utils/data/_utils/fetch.py", line 49, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/Users/cristianespinosa/opt/anaconda3/envs/tensor/lib/python3.9/site-packages/torch/utils/data/_utils/fetch.py", line 49, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched

In [ ]:
dataloader

In [17]:
counter = 0
g = 5
d = 1

for epoch in range(90):
    for batch in range(100):
        # After some training...
        if counter < d or counter >= (g+d):
            if counter >= (g+d):   #reset counter
                counter = 0
            print(epoch+1,') D is Training')


        elif counter < (g+d):
            print(epoch+1,') D is NOT Training')
    counter+=1

1 ) D is Training
1 ) D is Training
1 ) D is Training
1 ) D is Training
1 ) D is Training
1 ) D is Training
1 ) D is Training
1 ) D is Training
1 ) D is Training
1 ) D is Training
1 ) D is Training
1 ) D is Training
1 ) D is Training
1 ) D is Training
1 ) D is Training
1 ) D is Training
1 ) D is Training
1 ) D is Training
1 ) D is Training
1 ) D is Training
1 ) D is Training
1 ) D is Training
1 ) D is Training
1 ) D is Training
1 ) D is Training
1 ) D is Training
1 ) D is Training
1 ) D is Training
1 ) D is Training
1 ) D is Training
1 ) D is Training
1 ) D is Training
1 ) D is Training
1 ) D is Training
1 ) D is Training
1 ) D is Training
1 ) D is Training
1 ) D is Training
1 ) D is Training
1 ) D is Training
1 ) D is Training
1 ) D is Training
1 ) D is Training
1 ) D is Training
1 ) D is Training
1 ) D is Training
1 ) D is Training
1 ) D is Training
1 ) D is Training
1 ) D is Training
1 ) D is Training
1 ) D is Training
1 ) D is Training
1 ) D is Training
1 ) D is Training
1 ) D is T